In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

In [2]:
# Load data 
data = pd.read_csv('../../preprocessing/StorePreprocessed/GPT35csv.csv')



In [3]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["label"], test_size=0.3, random_state=42, stratify=data["label"])

In [4]:
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(4, 4), max_features=11000)  # dynamic range of features

In [5]:
# Remove NaN values from the split data
X_train = X_train.dropna()
X_test = X_test.dropna()
y_train = y_train[X_train.index]
y_test = y_test[X_test.index]

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [6]:
param_grid_rf = {
    'min_samples_split': [8, 32, 128],
    'max_features': [int(np.sqrt(X_train.shape[1])), int(0.02 * X_train.shape[1]), int(0.04 * X_train.shape[1]), int(0.06 * X_train.shape[1])]
}

In [7]:
rf = RandomForestClassifier(criterion='gini', oob_score=True)

In [8]:
# Grid search for Random Forest
grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy')
grid_rf.fit(X_train, y_train)

In [ ]:
# Evaluate Random Forest
rf_best = grid_rf.best_estimator_
y_pred_rf = rf_best.predict(X_test)
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
#0.85

Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.86      0.85      2754
           1       0.85      0.85      0.85      2754

    accuracy                           0.85      5508
   macro avg       0.85      0.85      0.85      5508
weighted avg       0.85      0.85      0.85      5508

Random Forest Accuracy: 0.8523965141612201
